In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=1afb2d712be040453b4bf3d846c1ebf47fc53400bc94d3a849e80402a3db6139
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyspark
from pyspark.sql.functions import row_number, col, count, sum as _sum, countDistinct
from pyspark.sql import Window
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import concat_ws, col
from pyspark.sql.functions import col, stddev, avg
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, FloatType
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/opt/conda/lib/python3.10/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ProductRecommendation") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/01 08:54:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.format("csv").option("header", True).load("/kaggle/input/amazon-ml-product-browse-node-classification/amazon dataset/amazon_train.csv")

In [5]:
df.show()

24/06/01 08:55:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
 Schema: _c0, TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
Expected: _c0 but found: 
CSV file: file:///kaggle/input/amazon-ml-product-browse-node-classification/amazon%20dataset/amazon_train.csv


+---+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0|               TITLE|         DESCRIPTION|       BULLET_POINTS|               BRAND|      BROWSE_NODE_ID|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|Pete The Cat Bedt...|Pete the Cat is t...|[Pete the Cat Bed...|         MerryMakers|                   0|
|  1|The New Yorker NY...|The New Yorker Ha...|[Cat In A Tea Cup...|      The New Yorker|                   1|
|  2|The Ultimate Self...|                NULL| Skyhorse Publishing|              imusti|                   2|
|  3|Amway Nutrilite K...|                NULL|[Nutrilite Kids,C...|               Amway|                   3|
|  4|Teacher Planner C...|                NULL|                NULL|                NULL|                   4|
|  5|Men'S Full Sleeve...|Men'S Full Sleeve...|[Color: Blue,Slee...|   Bhavya Enterprise|                   5|
|

**Preprocessing Trials**

In [6]:
df.printSchema()

root

 |-- _c0: string (nullable = true)

 |-- TITLE: string (nullable = true)

 |-- DESCRIPTION: string (nullable = true)

 |-- BULLET_POINTS: string (nullable = true)

 |-- BRAND: string (nullable = true)

 |-- BROWSE_NODE_ID: string (nullable = true)




In [7]:
# Perform the value counts equivalent
df_value_counts = df.groupBy("BROWSE_NODE_ID").agg(count("*").alias("count")).orderBy(col("count").desc())

# Show the result
df_value_counts.show()

+--------------+------+

|BROWSE_NODE_ID| count|

+--------------+------+

|          1045|102837|

|             5| 34524|

|          1251| 24573|

|          1052| 22699|

|             4| 17002|

|          1024| 16278|

|          1295| 15946|

|           374| 13935|

|           773| 13036|

|          1194| 12265|

|          1724| 11816|

|           701| 11777|

|            89|  9956|

|          1049|  9749|

|            11|  9170|

|           739|  8629|

|          1186|  8339|

|           803|  7965|

|          1687|  7946|

|            40|  7472|

+--------------+------+

only showing top 20 rows




In [8]:
# Perform the value counts equivalent
df_value_counts = df.groupBy("BRAND").agg(count("*").alias("count")).orderBy(col("count").desc())

# Show the result
df_value_counts.show()

+--------------------+-----+

|               BRAND|count|

+--------------------+-----+

|             Generic|51714|

|                NULL|30071|

|                Puma| 7226|

|United Colors of ...| 6150|

|                 Max| 5700|

| with print cover...| 5048|

| US Polo Association| 4383|

|              Adidas| 3835|

|                Nike| 3744|

|                 ACM| 3574|

|Amazon Brand - Sy...| 3324|

|           Shengshou| 3321|

|              Pikkme| 3211|

|      Tommy Hilfiger| 2618|

|        ELECTROPRIME| 2617|

|          Pepe Jeans| 2585|

|               GISMO| 2585|

|        ShopsGeniune| 2552|

|       Peter England| 2456|

|         Allen Solly| 2378|

+--------------------+-----+

only showing top 20 rows




In [9]:
# Filter the DataFrame
df_filtered = df.filter(col("BROWSE_NODE_ID") == 4)

# Define the window specification
window_spec = Window.orderBy("BROWSE_NODE_ID")

# Add row numbers to the DataFrame
df_with_row_numbers = df_filtered.withColumn("row_number", row_number().over(window_spec))

# Select the row where row_number is 162 (to get the 161st index in 0-based indexing)
selected_row = df_with_row_numbers.filter(col("row_number") == 162).select("TITLE").collect()

# Extract the TITLE value
if selected_row:
    title_value = selected_row[0]["TITLE"]
    print(title_value)
else:
    print("No such row found.")

Diameter31 inch,Printing Round Rug,Dragonfly,Mat Non-Slip Soft Entrance Mat Door Floor Rug Area Rug for Chair Living Room,，Compass Decor,Old Chart Ancient Map with Historical Territories Geographical Illustr


In [10]:
num_labels = df.select("BROWSE_NODE_ID").distinct().count()
print(f"Number of unique BROWSE_NODE_ID values: {num_labels}")

Number of unique BROWSE_NODE_ID values: 75988


In [11]:
num_labels = df.select("BRAND").distinct().count()
print(f"Number of unique BRAND values: {num_labels}")

Number of unique BRAND values: 241373


In [12]:
# Calculate the number of missing values for each column
missing_values_count = df.select([_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Show the result
missing_values_count.show()

+---+-----+-----------+-------------+-----+--------------+

|_c0|TITLE|DESCRIPTION|BULLET_POINTS|BRAND|BROWSE_NODE_ID|

+---+-----+-----------+-------------+-----+--------------+

|  0|   36|     371143|        86377|30071|           811|

+---+-----+-----------+-------------+-----+--------------+




In [13]:
# Filter the DataFrame to include only rows where 'BRAND' is null
BrandNA = df.filter(col("BRAND").isNull())

# Show the result
BrandNA.show()

+---+--------------------+--------------------+--------------------+-----+--------------+

|_c0|               TITLE|         DESCRIPTION|       BULLET_POINTS|BRAND|BROWSE_NODE_ID|

+---+--------------------+--------------------+--------------------+-----+--------------+

|  4|Teacher Planner C...|                NULL|                NULL| NULL|             4|

| 10|Slo-Niacin Polyge...|                NULL|                NULL| NULL|            10|

| 15|YASHICA Open Box ...|                NULL|[A precise and fa...| NULL|            15|

| 27|"ALL GLASS AQUARI...|These quality pin...|                NULL| NULL|            27|

| 47|       Susan Howatch|                NULL|                NULL| NULL|             4|

| 48|Elizabeth Reynold...|                NULL|                NULL| NULL|             4|

| 49|      Bohumil Hrabal|                NULL|                NULL| NULL|             4|

| 50|         E. B. White|                NULL|                NULL| NULL|             4|


**Count of missing Values with groupBy**

In [14]:
# Perform the value counts equivalent on the filtered DataFrame
browse_node_counts = BrandNA.groupBy("BROWSE_NODE_ID").agg(count("*").alias("count")).orderBy(col("count").desc())

# Show the result
browse_node_counts.show()

+--------------+-----+

|BROWSE_NODE_ID|count|

+--------------+-----+

|             4| 9123|

|            55|  770|

|            82|  362|

|           362|  341|

|          1762|  284|

|            69|  279|

|             0|  278|

|          1045|  235|

|          NULL|  227|

|            38|  203|

|           155|  160|

|           290|  157|

|           115|  148|

|           374|  141|

|           569|  134|

|           612|  133|

|            53|  124|

|           209|  115|

|            84|  105|

|            43|  102|

+--------------+-----+

only showing top 20 rows




In [16]:
# Define the columns to check for null values
columns_to_check = ['BULLET_POINTS', 'DESCRIPTION']

# Build the condition to filter rows where any of the specified columns are null
condition = col(columns_to_check[0]).isNull()
for column in columns_to_check[1:]:
    condition = condition | col(column).isNull()

# Filter the DataFrame based on the condition
null_rows = df.filter(condition)

# Show the result
null_rows.show()

+---+--------------------+--------------------+--------------------+--------------------+--------------+

|_c0|               TITLE|         DESCRIPTION|       BULLET_POINTS|               BRAND|BROWSE_NODE_ID|

+---+--------------------+--------------------+--------------------+--------------------+--------------+

|  2|The Ultimate Self...|                NULL| Skyhorse Publishing|              imusti|             2|

|  3|Amway Nutrilite K...|                NULL|[Nutrilite Kids,C...|               Amway|             3|

|  4|Teacher Planner C...|                NULL|                NULL|                NULL|             4|

|  9|Chapin 1380 Premi...|Chapin 1380 Premi...|                NULL|Chapin International|             9|

| 10|Slo-Niacin Polyge...|                NULL|                NULL|                NULL|            10|

| 12|Yamaha PCMSKIT Po...|                NULL|[Software teaches...|              YAMAHA|            12|

| 13|Microsoft Sidewin...|                NULL

In [17]:
# Perform the value counts equivalent on the filtered DataFrame
null_rows_counts = null_rows.groupBy("BROWSE_NODE_ID").agg(count("*").alias("count")).orderBy(col("count").desc())

# Show the result
null_rows_counts.show()

+--------------+-----+

|BROWSE_NODE_ID|count|

+--------------+-----+

|             5|13204|

|             4|11551|

|          1251|11198|

|          1024|10957|

|           701| 6734|

|            89| 6208|

|          1194| 5745|

|            40| 5443|

|          1254| 4931|

|           773| 4920|

|          1724| 4734|

|          1003| 4198|

|          1213| 4056|

|          1052| 3905|

|          1049| 3801|

|          1186| 3763|

|          1045| 3406|

|          2472| 3384|

|          1409| 2910|

|          1017| 2888|

+--------------+-----+

only showing top 20 rows




In [18]:
# Group by 'BRAND' and count unique 'BROWSE_NODE_ID' values
category_count = df.groupBy("BRAND").agg(countDistinct("BROWSE_NODE_ID").alias("unique_count"))

# Sort the result in descending order
category_count_sorted = category_count.orderBy(col("unique_count").desc())

# Show the result
category_count_sorted.show()


+------------+------------+

|       BRAND|unique_count|

+------------+------------+

|        NULL|        3853|

|     Generic|        3631|

|     Unknown|         792|

|ELECTROPRIME|         641|

|  FidgetGear|         536|

|       Leoie|         445|

|   Non-brand|         402|

|    Docooler|         398|

|      Anself|         334|

|    Magideal|         333|

|      uxcell|         321|

|      Disney|         296|

|    Prettyia|         291|

|       Homyl|         283|

|      Adidas|         279|

|         3nh|         275|

|      KKmoon|         261|

|        Nike|         253|

|    Segolike|         253|

|        Puma|         245|

+------------+------------+

only showing top 20 rows




In [19]:
# Filter the DataFrame for rows where the 'BRAND' column is equal to 'Unknown'
unknown_rows = df.filter(col("BRAND") == "Unknown")

# Show the result
unknown_rows.show()

+-----+--------------------+--------------------+--------------------+-------+--------------+

|  _c0|               TITLE|         DESCRIPTION|       BULLET_POINTS|  BRAND|BROWSE_NODE_ID|

+-----+--------------------+--------------------+--------------------+-------+--------------+

|  621|I Was Caught Doin...|Character pencils...|[I Was Caught Doi...|Unknown|           621|

|  913|SEA WATER PUMP JT...|JTP -10000 PUMPS ...|[PUMP FOR KOI PON...|Unknown|           272|

| 1628|Harry Potter Deat...|"Official License...|[Size: 9.5,Base M...|Unknown|           969|

| 2198|NEW Magnesium Cre...|NEW Magnesium Cre...|                NULL|Unknown|          2198|

| 2520|Homer Laughlin Ch...|Is it the bright,...|[Enter your model...|Unknown|          2520|

| 2916|Country Style Kit...|Features: <br> 10...|[Material: polyes...|Unknown|           739|

| 3099|LB1 High Performa...|<p>LB1 High Perfo...|[Turns One USB 3....|Unknown|          3099|

| 3218|Stylen21 Women's ...|Shop This Must-Ha...|[

In [20]:
# Filter the DataFrame for rows where the 'TITLE' column is null
null_title_rows = df.filter(col("TITLE").isNull())

# Show the result
null_title_rows.show()

+------+-----+--------------------+--------------------+--------------------+--------------------+

|   _c0|TITLE|         DESCRIPTION|       BULLET_POINTS|               BRAND|      BROWSE_NODE_ID|

+------+-----+--------------------+--------------------+--------------------+--------------------+

| 12687| NULL|Shop from a wide ...|[Fabric: 70% Cott...|                MANQ|                1254|

| 39229| NULL|-1 Heavy Copper W...|[-1 Heavy Copper ...|           DONGCHENG|                3230|

| 47841| NULL|This is a handcra...|                NULL|                NULL|                 918|

| 79427| NULL|Bra Extender give...|Bra Extender Come...|                 UHS|               28876|

|133807| NULL|U.S.Polo Associat...|[Wool,Slim Fit,So...| US Polo Association|                2181|

|142060| NULL|Shop from a wide ...|[Fabric: Polyeste...|       House of Pret|                1194|

|195257| NULL|Spruce up your wa...|[Fabric: Cotton; ...|      ANVI JEWELLERS|                1687|



In [35]:
df.show()

+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+----------+---------------+

|_c0|               TITLE|         DESCRIPTION|       BULLET_POINTS|               BRAND|      BROWSE_NODE_ID|word_count|char_count|sentiment_score|

+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+----------+---------------+

|  0|Pete The Cat Bedt...|Pete the Cat is t...|[Pete the Cat Bed...|         MerryMakers|                   0|         1|        11|         0.4019|

|  1|The New Yorker NY...|The New Yorker Ha...|[Cat In A Tea Cup...|      The New Yorker|                   1|         3|        14|            0.0|

|  2|The Ultimate Self...|                NULL| Skyhorse Publishing|              imusti|                   2|         1|         6|            0.0|

|  3|Amway Nutrilite K...|                NULL|[Nutrilite Kids,C...|               Amway|           

In [31]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, StopWordsRemover, FeatureHasher, VectorAssembler
from pyspark.ml import Pipeline

# Load and preprocess data
df = spark.read.csv("/kaggle/input/amazon-ml-product-browse-node-classification/amazon dataset/amazon_train.csv", header=True, inferSchema=True)
df = df.na.fill({"TITLE": "", "DESCRIPTION": "", "BULLET_POINTS": ""})
df = df.withColumn("text", concat_ws(" ", df.TITLE, df.DESCRIPTION, df.BULLET_POINTS))

df.show()

+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0|               TITLE|         DESCRIPTION|       BULLET_POINTS|               BRAND|      BROWSE_NODE_ID|                text|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|Pete The Cat Bedt...|Pete the Cat is t...|[Pete the Cat Bed...|         MerryMakers|                   0|Pete The Cat Bedt...|
|  1|The New Yorker NY...|The New Yorker Ha...|[Cat In A Tea Cup...|      The New Yorker|                   1|The New Yorker NY...|
|  2|The Ultimate Self...|                    | Skyhorse Publishing|              imusti|                   2|The Ultimate Self...|
|  3|Amway Nutrilite K...|                    |[Nutrilite Kids,C...|               Amway|                   3|Amway Nutrilite K...|
|  4|Teacher Planner C...|                    |                    |        

24/06/01 09:48:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
 Schema: _c0, TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
Expected: _c0 but found: 
CSV file: file:///kaggle/input/amazon-ml-product-browse-node-classification/amazon%20dataset/amazon_train.csv


In [32]:
# Tokenize the text data
tokenizer = Tokenizer(inputCol="text", outputCol="words")

# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filteredWords")



In [33]:
# Step 3: Compute TF-IDF for text features
hashingTF = HashingTF(inputCol="filteredWords", outputCol="rawFeatures", numFeatures=100)
idf = IDF(inputCol="rawFeatures", outputCol="textFeatures")

# Step 4: Feature hashing for BROWSE_NODE_ID with increased influence
# Increase the numFeatures to give more weight to BROWSE_NODE_ID
hasher = FeatureHasher(inputCols=["BROWSE_NODE_ID"], outputCol="categoryFeatures", numFeatures=30)

# Step 5: Combine text features and weighted category features
assembler = VectorAssembler(
    inputCols=["textFeatures", "categoryFeatures"],
    outputCol="features"
)

In [34]:
# Define the pipeline
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, hasher, assembler])

# Fit and transform the data
model = pipeline.fit(df)
finalData = model.transform(df)

In [35]:
finalData.show()

+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+
|_c0|               TITLE|         DESCRIPTION|       BULLET_POINTS|               BRAND|      BROWSE_NODE_ID|                text|               words|       filteredWords|         rawFeatures|        textFeatures|categoryFeatures|            features|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+
|  0|Pete The Cat Bedt...|Pete the Cat is t...|[Pete the Cat Bed...|         MerryMakers|                   0|Pete The Cat Bedt...|[pete, the, cat, ...|[pete, cat, bedti...|(100,[0,9,10,14,1...|(100,[0,9,10,14,1...| (30,[10],[1.0])|(130,[

24/06/01 09:50:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
 Schema: _c0, TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
Expected: _c0 but found: 
CSV file: file:///kaggle/input/amazon-ml-product-browse-node-classification/amazon%20dataset/amazon_train.csv


In [36]:
from pyspark.ml.feature import Normalizer

# Step 6: Normalize the features to improve the effectiveness of distance calculations
normalizer = Normalizer(inputCol="features", outputCol="normFeatures")
normalizedData = normalizer.transform(finalData)


In [37]:
from pyspark.ml.feature import BucketedRandomProjectionLSH

# Setup LSH model for fast approximate nearest neighbor search
brp = BucketedRandomProjectionLSH(
    inputCol="normFeatures", 
    outputCol="hashes", 
    bucketLength=2.0, 
    numHashTables=3
)
lshModel = brp.fit(normalizedData)


In [38]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

def recommend_products(product_id, num_recommendations=5):
    # Fetch the feature vector of the given product
    query_product = normalizedData.filter(f"_c0 = {product_id}").collect()
    if not query_product:
        raise ValueError("Product ID not found in the dataset.")
    
    # Extract the feature vector and browse ID
    query_features = query_product[0]['normFeatures']
    query_browse_id = query_product[0]['BROWSE_NODE_ID']
    
    # Filter out products with the same browse ID
    filtered_data = normalizedData.filter(col('BROWSE_NODE_ID') != query_browse_id)

    # Find approximate nearest neighbors; returns a dataframe with distances
    recommendations = lshModel.approxNearestNeighbors(
        filtered_data, query_features, num_recommendations, distCol="distCol"
    )

    # Add similarity percentage column
    def cosine_similarity(features):
        return float(query_features.dot(features) / (query_features.norm(2) * features.norm(2))) * 100

    cosine_similarity_udf = udf(cosine_similarity, FloatType())
    recommendations_with_similarity = recommendations.withColumn("similarity", cosine_similarity_udf("normFeatures"))

    return recommendations_with_similarity

In [39]:
# Example usage
product_id_example = 13  # Ensure this ID exists in your '_c0' column
recommendations_df = recommend_products(product_id_example, 5)
recommendations_df.select("_c0", "TITLE", "BROWSE_NODE_ID", "distCol", "similarity").show()

24/06/01 09:50:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
 Schema: _c0, TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
Expected: _c0 but found: 
CSV file: file:///kaggle/input/amazon-ml-product-browse-node-classification/amazon%20dataset/amazon_train.csv
24/06/01 09:52:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TITLE, DESCRIPTION, BULLET_POINTS, BROWSE_NODE_ID
 Schema: _c0, TITLE, DESCRIPTION, BULLET_POINTS, BROWSE_NODE_ID
Expected: _c0 but found: 
CSV file: file:///kaggle/input/amazon-ml-product-browse-node-classification/amazon%20dataset/amazon_train.csv
24/06/01 09:55:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TITLE, DESCRIPTION, BULLET_POINTS, BROWSE_NODE_ID
 Schema: _c0, TITLE, DESCRIPTION, BULLET_POINTS, BROWSE_NODE_ID
Expected: _c0 but found: 
CSV file: file:///kaggle/input/amazon-ml-product-browse-node-

+------+--------------------+--------------+------------------+----------+
|   _c0|               TITLE|BROWSE_NODE_ID|           distCol|similarity|
+------+--------------------+--------------+------------------+----------+
|499022|Tamatina Claude M...|           700|0.8107603538468586| 67.133385|
|846659|Lindo Microfiber ...|          3928|0.8278764110596661|  65.73103|
|672418|Tamatina Claude M...|           700|0.8374379183008279|  64.93488|
|364765|Kashi Micro-Suede...|   Linen Store|0.8411925259156107|  64.61976|
|813527|Pacifica Island A...|           413|0.8445312493706556|  64.33835|
+------+--------------------+--------------+------------------+----------+



In [40]:
from pyspark.sql.functions import col

def calculate_average_similarity(recommendations_df, feature_col='normFeatures'):
    # Assuming the query product features are included and is the first row in the DataFrame
    query_features = recommendations_df.head(1)[0][feature_col]
    
    # Calculate cosine similarity with the query product for each recommended product
    similarities = recommendations_df.rdd\
        .map(lambda x: (x[feature_col].dot(query_features) / (x[feature_col].norm(2) * query_features.norm(2))))\
        .collect()
    
    # Return the average similarity
    return sum(similarities) / len(similarities)

# Use the function
average_similarity = calculate_average_similarity(recommendations_df)
print("Average Cosine Similarity:", average_similarity)


24/06/01 09:58:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
 Schema: _c0, TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
Expected: _c0 but found: 
CSV file: file:///kaggle/input/amazon-ml-product-browse-node-classification/amazon%20dataset/amazon_train.csv
24/06/01 10:01:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
 Schema: _c0, TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
Expected: _c0 but found: 
CSV file: file:///kaggle/input/amazon-ml-product-browse-node-classification/amazon%20dataset/amazon_train.csv
24/06/01 10:04:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
 Schema: _c0, TITLE, DESCRIPTION, BULLET_POINTS, BRAND, BROWSE_NODE_ID
Expected: _c0 but found: 
CSV file: file:///kaggle/input/am

Average Cosine Similarity: 0.7842110689415577


In [ ]:
# Stop Spark session
spark.stop()